In [2]:
import os
os.system('pip install boto3')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 17.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.41.5
    Uninstalling botocore-1.41.5:
      Successfully uninstalled botocore-1.41.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.26.0 requires botocore<1.41.6,>=1.41.0, but you have botocore 1.42.9 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


0

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
# import boto3
from datetime import datetime

In [2]:
spark = SparkSession.builder \
    .appName("ingestion_bronze") \
    .config("spark.cores.max", "1") \
    .config("spark.executor.memory", "2g") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hive-metastore:9083") \
    .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/") \
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/23 17:34:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
categories = '../data/categories_full.csv'
df = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'false') \
    .option("escape", '"') \
    .option("quote", '"') \
    .option("multiLine", "true") \
    .option("ignoreLeadingWhiteSpace", "true") \
    .option("ignoreTrailingWhiteSpace", "true") \
    .load(categories)
# df.printSchema()
# df.show(5)

df = df.withColumn('ngay_cap_nhat', current_timestamp())
df.write \
    .format('iceberg') \
    .mode('overwrite') \
    .saveAsTable('iceberg.bronze.categories')

In [4]:
sub_categories = '../data/sub_categories.csv'
df = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'false') \
    .option("escape", '"') \
    .option("quote", '"') \
    .option("multiLine", "true") \
    .option("ignoreLeadingWhiteSpace", "true") \
    .option("ignoreTrailingWhiteSpace", "true") \
    .load(sub_categories)
# df.printSchema()
# df.show(5)

df = df.withColumn('ngay_cap_nhat', current_timestamp())
df.write \
    .format('iceberg') \
    .mode('overwrite') \
    .saveAsTable('iceberg.bronze.sub_categories')

In [5]:
product_category = '../data/all_product_ids.csv'
df = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'false') \
    .option("escape", '"') \
    .option("quote", '"') \
    .option("multiLine", "true") \
    .option("ignoreLeadingWhiteSpace", "true") \
    .option("ignoreTrailingWhiteSpace", "true") \
    .load(product_category)
# df.printSchema()
# df.show(5)

df = df.withColumn('ngay_cap_nhat', current_timestamp())
df.write \
    .format('iceberg') \
    .mode('overwrite') \
    .saveAsTable('iceberg.bronze.product_category')

In [6]:
product_details = '../data/product_details.csv'
df = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'false') \
    .option("escape", '"') \
    .option("quote", '"') \
    .option("multiLine", "true") \
    .option("ignoreLeadingWhiteSpace", "true") \
    .option("ignoreTrailingWhiteSpace", "true") \
    .load(product_details)
# df.printSchema()
# df.show(5)

df = df.withColumn('ngay_cap_nhat', current_timestamp())
df.write \
    .format('iceberg') \
    .mode('overwrite') \
    .saveAsTable('iceberg.bronze.product_details')

In [7]:
reviews = '../data/all_reviews_parallel.csv'
df = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'false') \
    .option("escape", '"') \
    .option("quote", '"') \
    .option("multiLine", "true") \
    .option("ignoreLeadingWhiteSpace", "true") \
    .option("ignoreTrailingWhiteSpace", "true") \
    .load(reviews)
# df.printSchema()
# df.show(5)

df = df.withColumn('ngay_cap_nhat', current_timestamp())
df.write \
    .format('iceberg') \
    .mode('overwrite') \
    .saveAsTable('iceberg.bronze.reviews')

In [8]:
spark.stop()